<a href="https://colab.research.google.com/github/jensullrich/DataScienceSS20/blob/ex7/Exercises/7_b_CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 2 - Image Classification with CNNs
![CIFAR10](https://miro.medium.com/max/964/1*syyml8q8s1Yt-iEea5m1Ag.png)

[CIFAR 10](https://www.cs.toronto.edu/~kriz/cifar.html) is a small toy data set for image classifaction. 

* It has 60k images from ten diffenrent classes (like car, airplane, horse,..)
* The RGB images are of size 32x32 pixels  

In [25]:
#import needed libs
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import os

#some vars to controll the training -> use them in your code
batch_size = 64
num_classes = 10 #fixed!
epochs = 15

# Get the data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## Task:
* build and train the ***AlexNet*** CNN architecture from the lecture
* use 'relu' functions for the non-linear activation
* optimize the learning hyper-paramters to get the best  test accuracy

In [26]:
# build model
#Instantiate an empty model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=48, input_shape=(32,32,3), kernel_size=(3, 3),\
 strides=(4,4), padding='same'))
model.add(Activation('relu'))
# Pooling 
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=192, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

# 4th Convolutional Layer
model.add(Conv2D(filters=192, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

# 5th Convolutional Layer
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(1024, input_shape=(32*32*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))

# 2nd Dense Layer
model.add(Dense(1024))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# 3rd Dense Layer
model.add(Dense(512))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(num_classes))
model.add(Activation('softmax'))

opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 8, 8, 48)          1344      
_________________________________________________________________
activation_64 (Activation)   (None, 8, 8, 48)          0         
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 4, 4, 48)          0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 4, 4, 48)          192       
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 4, 4, 128)         55424     
_________________________________________________________________
activation_65 (Activation)   (None, 4, 4, 128)         0         
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 2, 2, 128)        

In [27]:
#pre-processing: normalize data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
#train model
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              shuffle=True)

Epoch 1/15
50000/50000 [==============================] - 21s 422us/step - loss: 1.7243 - accuracy: 0.3665
Epoch 2/15
50000/50000 [==============================] - 21s 410us/step - loss: 1.4013 - accuracy: 0.4976
Epoch 3/15
50000/50000 [==============================] - 20s 406us/step - loss: 1.2584 - accuracy: 0.5535
Epoch 4/15
50000/50000 [==============================] - 21s 411us/step - loss: 1.1474 - accuracy: 0.5950
Epoch 5/15
50000/50000 [==============================] - 20s 407us/step - loss: 1.0495 - accuracy: 0.6296
Epoch 6/15
50000/50000 [==============================] - 21s 411us/step - loss: 0.9498 - accuracy: 0.6672
Epoch 7/15
50000/50000 [==============================] - 21s 411us/step - loss: 0.8584 - accuracy: 0.7008
Epoch 8/15
50000/50000 [==============================] - 20s 410us/step - loss: 0.7619 - accuracy: 0.7341
Epoch 9/15
50000/50000 [==============================] - 20s 407us/step - loss: 0.6791 - accuracy: 0.7645
Epoch 10/15
50000/50000 [============

In [28]:
#Evaluation
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 224us/step
Test loss: 2.039508597946167
Test accuracy: 0.5540000200271606
